In [1]:
import os
import sys
import csv
import pandas as pd
import numpy as np

import os.path
from os import path

import openslide
from skimage.color import rgb2hsv
import PIL
from PIL import Image

import cv2
import glob
import h5py
import os
import pathlib
import tables

import torch
from skimage import io, transform

import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import timm
import torch.nn as nn

In [2]:
#manifest txt to dataframe LUAD
luaddf = pd.read_table('/home/irene/Downloads/gdc_manifest_20220108_143759.txt', delimiter = '\t',
                    header = 'infer')
print(len(luaddf))
print(luaddf)
print(luaddf.columns.values)
print(luaddf.loc[2, 'id'])

541
                                       id  \
0    004d8238-0a74-40bd-9547-f48a2086c416   
1    00a0b174-1eab-446a-ba8c-7c6e3acd7f0c   
2    00efe162-75c1-4451-814c-1576cf6856f8   
3    015862cf-a450-48fa-b987-f993729436a6   
4    01954393-6fd5-422b-bf9e-36fab4452619   
..                                    ...   
536  fc0fe5e3-aa78-4652-aeb2-fd73b46b7b5b   
537  fc7d3a58-119d-4cb5-9b48-1932c9f271e9   
538  fc83476f-eb45-46db-b87b-25225d4417d0   
539  fd1c95ff-26b9-4a95-a671-fa1e2b511cbe   
540  fdb89f0d-4402-493a-a6ef-a88cfb9542e8   

                                              filename  \
0    TCGA-75-7030-01Z-00-DX1.5DDF24B5-00D1-4418-A06...   
1    TCGA-MN-A4N4-01Z-00-DX2.9550732D-8FB1-43D9-B09...   
2    TCGA-86-6562-01Z-00-DX1.5dea3015-e606-4837-9f9...   
3    TCGA-97-7552-01Z-00-DX1.a5ab8959-ff7a-4287-b01...   
4    TCGA-38-6178-01Z-00-DX1.5932337D-BA2B-455D-9E3...   
..                                                 ...   
536  TCGA-55-7816-01Z-00-DX1.A58C6196-E1A0-48B3-9

In [3]:

luscdf = pd.read_table('/home/irene/Downloads/gdc_manifest_20220107_164604.txt', delimiter = '\t',
                    header = 'infer')
print(len(luscdf))
print(luscdf)
print(luscdf.columns.values)
print(luscdf.loc[2, 'id'])

512
                                       id  \
0    00d7bab3-386e-413a-a3be-e1ec312b1754   
1    01a0a249-0ff2-492f-bab7-20121922906b   
2    028c2af5-9ab1-4822-ae45-2fd870c91879   
3    0328b5b6-47be-44d5-bb9a-872637562bc5   
4    033ca180-8ae6-4219-9978-29c828ea3ac3   
..                                    ...   
507  fd9b912b-7206-4eb1-980e-997c5c7dd63f   
508  fdee3a3e-ff61-41b9-9925-90fba0dbd063   
509  fe2cf123-87bc-44af-8240-5e1f830ddca1   
510  feaf4455-0ea2-4b0b-8b58-528e330354ad   
511  ff85d168-a20c-467a-9edf-cf9ad517c421   

                                              filename  \
0    TCGA-56-8503-01Z-00-DX1.87FEB654-FC88-4CDA-8F5...   
1    TCGA-77-8138-01Z-00-DX1.fac6dcf9-7367-4345-a76...   
2    TCGA-98-A53H-01Z-00-DX1.B05E6339-440C-4F2E-95B...   
3    TCGA-43-A56V-01Z-00-DX1.AA93FE03-FA7D-42C4-A11...   
4    TCGA-56-A49D-01Z-00-DX1.1ECF40A0-1490-42B3-892...   
..                                                 ...   
507  TCGA-22-5473-01Z-00-DX1.C452FE31-443B-43AE-A

In [4]:
#check if the image folder path exists in the directory
all_pathad = []
exist_pathad = []
exist_idad = []
exist_svsad = []
count = 0
for i in range(len(luaddf)):
    folder_path = luaddf.loc[i, 'id']
    svs_path = luaddf.loc[i, 'filename']
    all_pathad.append(folder_path + "/" + svs_path)
    if(os.path.isfile('/home/irene/Downloads/TCGA-LUAD/'+folder_path + "/" + svs_path)):
        exist_pathad.append('/home/irene/Downloads/TCGA-LUAD/'+folder_path + "/" + svs_path)
        print('/home/irene/Downloads/TCGA-LUAD/'+folder_path + "/" + svs_path)
        print(count)
        exist_idad.append(folder_path)
        exist_svsad.append(svs_path)
        count = count + 1
print(len(exist_pathad))
print(len(all_pathad))

/home/irene/Downloads/TCGA-LUAD/00a0b174-1eab-446a-ba8c-7c6e3acd7f0c/TCGA-MN-A4N4-01Z-00-DX2.9550732D-8FB1-43D9-B094-7C0CD310E9C0.svs
0
/home/irene/Downloads/TCGA-LUAD/00efe162-75c1-4451-814c-1576cf6856f8/TCGA-86-6562-01Z-00-DX1.5dea3015-e606-4837-9f99-ac14f0aa091b.svs
1
/home/irene/Downloads/TCGA-LUAD/022450db-7064-4a04-ba2d-98d0a67fb84e/TCGA-49-AARQ-01Z-00-DX1.3FA36419-1658-4602-9170-84A622787C3E.svs
2
/home/irene/Downloads/TCGA-LUAD/03309e6b-eb69-4009-9f7b-74b0a0f00de4/TCGA-55-7284-01Z-00-DX1.68b95b9b-1aab-4f03-aad3-1132467b7499.svs
3
/home/irene/Downloads/TCGA-LUAD/065e8ed3-adce-477d-8012-6cfc498bc335/TCGA-69-8453-01Z-00-DX1.C472096B-95BA-42CD-ADEB-326C38F9DC95.svs
4
/home/irene/Downloads/TCGA-LUAD/09acdcf2-f634-452f-b78b-975128e7d14b/TCGA-93-8067-01Z-00-DX1.325d0cd5-9bb5-4f51-a197-75d3c3d19aaf.svs
5
/home/irene/Downloads/TCGA-LUAD/0e64c462-0c17-4e84-bb04-96976f01f62b/TCGA-55-8089-01Z-00-DX1.da4d99ff-4a7c-45a3-b79c-039c0c9e9712.svs
6
/home/irene/Downloads/TCGA-LUAD/100a645f-75a4-4c

In [5]:

all_pathsc =[]
exist_pathsc =[]
exist_idsc =[]
exist_svssc = []
count = 0
for i in range(len(luscdf)):
    folder_path = luscdf.loc[i, 'id']
    svs_path = luscdf.loc[i, 'filename']
    all_pathsc.append(folder_path + "/" + svs_path)
    if(os.path.isfile('/home/irene/Downloads/TCGA-LUSC/'+folder_path + "/" + svs_path)):
        exist_pathsc.append('/home/irene/Downloads/TCGA-LUSC/'+folder_path + "/" + svs_path)
        print('/home/irene/Downloads/TCGA-LUSC/'+folder_path + "/" + svs_path)
        print(count)
        exist_idsc.append(folder_path)
        exist_svssc.append(svs_path)
        count = count + 1
print(len(exist_pathsc))
print(len(all_pathsc))

/home/irene/Downloads/TCGA-LUSC/0439d959-547f-4299-a962-c659dc88f216/TCGA-66-2771-01Z-00-DX1.b84a28ec-2726-4386-9445-0f5ca351d985.svs
0
/home/irene/Downloads/TCGA-LUSC/093cd40c-09d4-4d21-b97c-84be9327f3fd/TCGA-33-4583-01Z-00-DX3.f802363d-0e77-450e-bf45-928629c66e23.svs
1
/home/irene/Downloads/TCGA-LUSC/0cf6950d-4c6b-4a86-87fd-b2ed121db107/TCGA-51-4079-01Z-00-DX1.111f7796-3797-4334-b543-918e374dbc22.svs
2
/home/irene/Downloads/TCGA-LUSC/0e226b74-1cde-4bb5-9998-2e94a7fe16ba/TCGA-85-8352-01Z-00-DX1.47426016-95BC-49C0-9844-06A3A7EAE5D6.svs
3
/home/irene/Downloads/TCGA-LUSC/10afc57b-4051-4f82-a41d-738f06706044/TCGA-43-2578-01Z-00-DX1.3314c57b-2875-4cb3-8fff-cb9cf65aa697.svs
4
/home/irene/Downloads/TCGA-LUSC/1123d866-9b24-47e1-a285-20a374d62df0/TCGA-60-2712-01Z-00-DX1.97003dfc-4b37-4491-86e8-0801e6825aae.svs
5
/home/irene/Downloads/TCGA-LUSC/18cd3aed-514e-4b56-9c52-04c06d83eaa7/TCGA-98-7454-01Z-00-DX1.2d81415b-5edd-4dd8-b17b-849d732da4cd.svs
6
/home/irene/Downloads/TCGA-LUSC/1a0c639a-e273-41

In [6]:
luad_list = glob.glob('/home/irene/Downloads/LUAD-image/**/**')
lusc_list = glob.glob('/home/irene/Downloads/LUSC-image/**/**')
luad_list.sort()
lusc_list.sort()
labels = [0]*len(luad_list)
labels.extend([1]*len(lusc_list))
path_list = luad_list+lusc_list

In [7]:
idlist = []
for i in range (len(luad_list)):
    string = luad_list[i]
    idlist.append(string[33:69])
for i in range (len(lusc_list)):
    string = lusc_list[i]
    idlist.append(string[33:69])

In [8]:
df = pd.DataFrame({'image_path': path_list, 'label': labels, 'id':idlist})
df

,image_path,label,id
0,/home/irene/Downloads/LUAD-image/00a0b174-1eab...,0,00a0b174-1eab-446a-ba8c-7c6e3acd7f0c
1,/home/irene/Downloads/LUAD-image/00a0b174-1eab...,0,00a0b174-1eab-446a-ba8c-7c6e3acd7f0c
2,/home/irene/Downloads/LUAD-image/00a0b174-1eab...,0,00a0b174-1eab-446a-ba8c-7c6e3acd7f0c
3,/home/irene/Downloads/LUAD-image/00a0b174-1eab...,0,00a0b174-1eab-446a-ba8c-7c6e3acd7f0c
4,/home/irene/Downloads/LUAD-image/00a0b174-1eab...,0,00a0b174-1eab-446a-ba8c-7c6e3acd7f0c
...,...,...,...
49641,/home/irene/Downloads/LUSC-image/fcd48b49-680a...,1,fcd48b49-680a-4144-8817-4a527f2740cc
49642,/home/irene/Downloads/LUSC-image/fcd48b49-680a...,1,fcd48b49-680a-4144-8817-4a527f2740cc
49643,/home/irene/Downloads/LUSC-image/fcd48b49-680a...,1,fcd48b49-680a-4144-8817-4a527f2740cc
49644,/home/irene/Downloads/LUSC-image/fcd48b49-680a...,1,fcd48b49-680a-4144-8817-4a527f2740cc


In [9]:
df.to_csv(r'/home/irene/Downloads/datapath.csv')

In [10]:
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, test_size = 0.4, stratify = df['id'], random_state = 0)
val_df, test_df = train_test_split(temp_df, test_size = 0.5, stratify = temp_df['id'], random_state = 0)

In [11]:
test_df

,image_path,label,id
31941,/home/irene/Downloads/LUSC-image/53adb84c-8f77...,1,53adb84c-8f77-4786-94f4-49000e4bb2fe
13001,/home/irene/Downloads/LUAD-image/67a048bd-9130...,0,67a048bd-9130-4662-b6ff-5badad42cae9
20442,/home/irene/Downloads/LUAD-image/c76ed21f-f466...,0,c76ed21f-f466-452e-90a0-e73726c6a13e
18207,/home/irene/Downloads/LUAD-image/a28f5d37-6ab4...,0,a28f5d37-6ab4-4567-b945-79c73048ac5f
10971,/home/irene/Downloads/LUAD-image/57246c93-5967...,0,57246c93-5967-4f70-a29e-85d0e325566c
...,...,...,...
47450,/home/irene/Downloads/LUSC-image/e63b3a58-4ff8...,1,e63b3a58-4ff8-499b-a42b-f35cfc86cd0f
30074,/home/irene/Downloads/LUSC-image/496f1be5-1c5d...,1,496f1be5-1c5d-464c-b3e5-ea5ab1d5f58b
33750,/home/irene/Downloads/LUSC-image/691c34ee-8b95...,1,691c34ee-8b95-4199-83e2-624a9e59470f
34084,/home/irene/Downloads/LUSC-image/6bc57f51-fffa...,1,6bc57f51-fffa-4f82-9966-7547d65ad8b0


In [12]:
img_dtype = tables.UInt8Atom()

In [15]:
data_shape= (0,224,224,4)

h5_path = '/home/irene/Downloads/luadlusc.hdf5'

h5_file = tables.open_file(h5_path,mode='w')

train_storage = h5_file.create_earray(h5_file.root, 'train_img',img_dtype, shape = data_shape) 
val_storage   = h5_file.create_earray(h5_file.root, 'val_img',img_dtype, shape = data_shape) 
test_storage  = h5_file.create_earray(h5_file.root, 'test_img',img_dtype, shape = data_shape) 

train_img_list = train_df['image_path'].to_list()
val_img_list = val_df['image_path'].to_list()
test_img_list = test_df['image_path'].to_list()

h5_file.create_array(h5_file.root,'train_label',train_df['label'].to_list())
h5_file.create_array(h5_file.root,'val_label',val_df['label'].to_list())
h5_file.create_array(h5_file.root,'test_label',test_df['label'].to_list())
#order of code is important
for i in range(len(train_img_list)):
    path = train_img_list[i]
    img = Image.open(path)
    img = img.resize((224,224),resample = Image.BICUBIC)
    img_arr = np.asarray(img)
    print(path)
    print(img_arr.shape)
    train_storage.append(img_arr[None])
for i in range(len(val_img_list)):
    addr = val_img_list[i]
    img = Image.open(addr)
    img = img.resize((224, 224), resample=Image.BICUBIC)
    img_arr = np.asarray(img)
    val_storage.append(img_arr[None])

for i in range(len(test_img_list)):
    addr = test_img_list[i]
    img = Image.open(addr)
    img = img.resize((224, 224), resample=Image.BICUBIC)
    img_arr = np.asarray(img)
    test_storage.append(img_arr[None])



/home/irene/Downloads/LUSC-image/c2162929-572d-45f3-b211-6f0b54d61650/30000_11000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/8b034336-e045-4fba-8ea6-406b68b9d98c/112000_29000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/9a077587-a524-4622-afbc-c061b91cfcaf/86000_25000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/e5def156-4f08-4e67-8573-7ba728f51675/85000_48000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/d02dc78f-d587-4614-91e9-694da76c906e/97000_57000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/d190a72b-c458-4791-b9dc-ff8b4fa481dc/14000_23000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/9e0bbb1b-5f03-4b95-a885-5f455fc73f8e/6000_9000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/c2162929-572d-45f3-b211-6f0b54d61650/39000_7000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/8da033c6-3011-4568-988d-13c394038920/22000_34000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/d5ac2dcf-a3a2-44a3-b418-0fab642e3b7f/22000_21000.png
(224, 224, 4)
/h

/home/irene/Downloads/LUSC-image/335772a1-45a4-4e08-b26a-bfe3165e6e8c/54000_16000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/96850824-2e13-4bf8-af9e-cab1c0e958d8/30000_58000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/b6fea18e-9615-49c1-a10b-1d466639c2bd/20000_11000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/0e226b74-1cde-4bb5-9998-2e94a7fe16ba/20000_31000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/26ad2684-4cc8-4191-841c-42b8f170c7e0/67000_32000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/b49d7f7e-1e12-47d7-93d1-0c95e0967952/132000_11000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/8b034336-e045-4fba-8ea6-406b68b9d98c/95000_38000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/4a79f19b-3a67-4ce2-994c-7cd4e29979c7/60000_46000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/6cec8838-5a03-4fca-a9b6-bfc5e4ef4fe6/39000_41000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/4daa3fa7-5336-4b51-a8c6-7762e89d2011/21000_50000.png
(224, 224, 4)

/home/irene/Downloads/LUAD-image/5389e7cf-ca61-4264-b912-9b802c5b1165/11000_31000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/46aaae5a-36d4-4de0-9dc4-12313c0dfeeb/5000_27000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/5389e7cf-ca61-4264-b912-9b802c5b1165/7000_36000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/e63b3a58-4ff8-499b-a42b-f35cfc86cd0f/22000_37000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/e9858b22-f489-485a-8a77-a320b0a81b13/76000_55000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/640fbebc-6d08-4d4a-bbc8-5d6d73626883/52000_32000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/d190a72b-c458-4791-b9dc-ff8b4fa481dc/23000_25000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/9844f974-7ee0-42da-9ee6-6213921ecf64/46000_20000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/46373e9f-11c3-40d1-aa12-b615af7fd10c/119000_27000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/f9e6881f-0ceb-4702-b72d-c3df13972ab0/20000_6000.png
(224, 224, 4)
/h

/home/irene/Downloads/LUAD-image/5d220819-58aa-4c3b-afe0-b442ebdd53a1/9000_4000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/f96a9f38-52c5-4736-8d00-c30e0b3cedf8/43000_50000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/2d0a8670-7dfc-4a46-b570-e435cbdfc6fb/87000_51000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/54e51b84-73f7-43d1-87e8-aa91e44205d9/41000_8000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/c2162929-572d-45f3-b211-6f0b54d61650/6000_6000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/d5d00824-8f67-4a9d-b772-71569d3659c5/80000_20000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/2d0a8670-7dfc-4a46-b570-e435cbdfc6fb/88000_55000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/1c954257-dff9-4137-b6e4-c92ca524c6f9/62000_16000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/e1c4b4a1-7c62-46ef-b406-d56bca4aa950/48000_24000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/9dce4938-27d5-4926-9c66-7176369d27e8/20000_24000.png
(224, 224, 4)
/home

/home/irene/Downloads/LUSC-image/c863b3b5-ebd2-43b6-99d6-d5741e872ff3/14000_5000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/aaa30733-9c20-4f3e-a42c-847bd080c417/61000_18000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/458169a9-fb2f-4c27-84cd-6f1222f3c56d/35000_58000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/c21f0d2c-73c1-463e-b6d2-94746a048b55/18000_31000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/3614fe76-11a3-4b95-a0d6-426ec32fe90c/12000_4000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/47176200-f735-44f4-97a2-9f79914a7d3a/59000_13000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/2234d946-a73c-4a9d-bbde-d55fb4495680/52000_52000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/60f8249b-abf8-4ae3-a472-ed2a428e3834/62000_23000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/98dd0957-8cbe-48e3-931a-76a58706d9b8/3000_29000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/67a048bd-9130-4662-b6ff-5badad42cae9/38000_61000.png
(224, 224, 4)
/ho

...

/home/irene/Downloads/LUSC-image/e63b3a58-4ff8-499b-a42b-f35cfc86cd0f/12000_21000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/42b16372-261d-42d2-95e5-27cae585e69d/15000_23000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/1a0c639a-e273-41ba-88dd-9da758cfb85d/17000_6000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/d77790d0-7556-4d46-8832-ece604082fd5/81000_49000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/32afadea-50d6-4df1-95da-14b3aa829aac/50000_24000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/4bf5d80f-af5f-44a5-b3b2-d0f35a9284de/102000_20000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/50178e6d-ee5f-4200-b0b1-faeab97b2c1b/36000_56000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/1616d5de-a80a-47e8-9ad6-1e22097c5633/48000_40000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/817a653b-f4bc-4839-a93f-e1ce19bd3e8d/28000_19000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/71297f8e-34e5-42d7-bbab-78f0c356388c/20000_35000.png
(224, 224, 4)


/home/irene/Downloads/LUSC-image/ddc21b43-4b22-40f7-8faa-b1a464a3fbed/77000_39000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/c57d543d-732a-437d-9ec0-6e07834fa227/101000_23000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/1a0c639a-e273-41ba-88dd-9da758cfb85d/10000_8000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/5192c4d5-69df-400e-af31-c964c9512bd7/49000_6000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/99a6abcb-b504-4478-bbf0-7b0d00820171/62000_48000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/6d2f7690-568e-49b5-800a-a2ad39737bfb/66000_41000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/fcd48b49-680a-4144-8817-4a527f2740cc/31000_61000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/99a6abcb-b504-4478-bbf0-7b0d00820171/104000_32000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/d77790d0-7556-4d46-8832-ece604082fd5/31000_10000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/e63b3a58-4ff8-499b-a42b-f35cfc86cd0f/23000_22000.png
(224, 224, 4)


/home/irene/Downloads/LUAD-image/30489618-b4bf-404c-8a3a-98c9b18bc5ec/19000_49000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/f28b9a47-68b8-4549-abfd-06e3038dad7b/15000_37000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/1b9b90db-2916-45f2-b945-a7ea1196a6de/69000_40000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/eafb7808-433d-4614-b5aa-6fd6167e0c47/106000_16000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/50178e6d-ee5f-4200-b0b1-faeab97b2c1b/30000_53000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/472ab102-7006-481e-a94f-c33da6a82689/17000_4000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/3f354f22-d0ef-43c4-880e-01e77c0bf4b8/28000_41000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/c521d208-41ba-4df5-81e5-a199bba27c2b/54000_11000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/3614fe76-11a3-4b95-a0d6-426ec32fe90c/11000_9000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/5f739d40-1c4f-47f4-87d1-4368e8b6b0b9/18000_12000.png
(224, 224, 4)
/

/home/irene/Downloads/LUSC-image/6d2f7690-568e-49b5-800a-a2ad39737bfb/52000_35000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/32afadea-50d6-4df1-95da-14b3aa829aac/69000_26000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/1e894464-0639-4fd7-8375-71b809ada590/77000_7000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/1c954257-dff9-4137-b6e4-c92ca524c6f9/60000_24000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/c12c8b4c-4733-4dd9-867d-3c4552684907/39000_15000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/c21f0d2c-73c1-463e-b6d2-94746a048b55/31000_27000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/393bfe7f-3c71-41b8-b608-723d65d7dad7/16000_14000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/9db4a2ba-2e3e-4c8e-a189-1c50d0598c54/58000_46000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/5389e7cf-ca61-4264-b912-9b802c5b1165/21000_12000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/53adb84c-8f77-4786-94f4-49000e4bb2fe/16000_31000.png
(224, 224, 4)
/

/home/irene/Downloads/LUSC-image/2285b642-ac05-4e32-8640-f81be14b4a05/54000_32000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/4d231d26-9b71-489d-9436-1186b88f7d47/12000_6000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/4cae8c1a-195a-49d3-a88e-f863c6506ab2/12000_24000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/530ffaf5-0e2a-44f7-9c84-75199c9799fd/68000_45000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/d5d00824-8f67-4a9d-b772-71569d3659c5/78000_24000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/c521d208-41ba-4df5-81e5-a199bba27c2b/2000_1000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/00a0b174-1eab-446a-ba8c-7c6e3acd7f0c/26000_33000.png
(224, 224, 4)
/home/irene/Downloads/LUSC-image/5ca1e5a0-1df9-4261-9ab2-ebb9a8d85d95/82000_31000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/8fc70255-9886-435f-9d69-7374aeb15c0f/24000_66000.png
(224, 224, 4)
/home/irene/Downloads/LUAD-image/1d896565-f1cc-4e6f-9197-41642e3e4ac8/34000_23000.png
(224, 224, 4)
/ho

In [16]:
h5_file.close()

In [17]:
hdf5_path = '/home/irene/Downloads/luadlusc.hdf5'
file = h5py.File(hdf5_path, "r")
file.keys()

<KeysViewHDF5 ['test_img', 'test_label', 'train_img', 'train_label', 'val_img', 'val_label']>

In [18]:
rgb_mean,rgb_std = (0.485, 0.456, 0.406),(0.229, 0.224, 0.225)

In [19]:
class HDF5Dataset(Dataset):


    def __init__(self, h5_path, set_name, transform = None):
        self.h5_path = h5_path
        self.dataset = None
        self.transform = transform
        self.file_path  = h5_path
        self.set = set_name
        
        str_name = self.set + "_img"
        
        file = h5py.File(h5_path, "r")
        self.dataset_len = len(file[str_name])
        
        self.transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize(rgb_mean,rgb_std)
            ])
        
    def __getitem__(self, index): #to enable indexing
        
        if self.dataset is None:
            self.imgs = h5py.File(self.file_path, 'r')[self.set + "_img"]
            self.labels = h5py.File(self.file_path, 'r')[self.set + "_label"]
            
            cur_img = self.imgs[index]
            PIL_image = Image.fromarray(np.uint8(cur_img)).convert('RGB')
            image = self.transform(PIL_image)
            label = self.labels[index].astype('float32')
            
            
        return (image,label)

    def __len__(self):
        return self.dataset_len

In [20]:
from torch.utils.data import Dataset,DataLoader

train_loader = DataLoader(HDF5Dataset(h5_path=hdf5_path,set_name="train"), batch_size=64,shuffle=True,drop_last=False)
val_loader = DataLoader(HDF5Dataset(h5_path=hdf5_path,set_name="val"), batch_size=64,shuffle=True,drop_last=False)
test_loader = DataLoader(HDF5Dataset(h5_path=hdf5_path,set_name="test"), batch_size=64,shuffle=True,drop_last=False)

In [21]:
model = timm.create_model('resnet50',pretrained=True)
model.fc = nn.Linear(model.fc.in_features,2)

loaders = {'train':train_loader,'val':val_loader,'test':test_loader}